## LSTMによる分類
torch.nn.Embeddingとtorch.nn.LSTMを利用 <br />
<br />
<img src="figures/LSTM.jpg" width="320px" align="left"><br clear="all" />
<br />
上記は1データ（1系列）を入力する場合のイメージ．<br />
以下では，複数のデータをまとめて一つのミニバッチとして入力する場合の実装を示す． <br />

### データの読み込みと整形
単語分割 -> 各単語をidに変換

In [59]:
from data import Corpus # 自作クラス
dataset = Corpus("text_data.txt")
vocab_size = len(dataset.word_to_id_dict) # 総語彙数 (入力サイズ)
output_size = len(dataset.label_to_id_dict) # ラベル数 (出力サイズ)

In [60]:
input_sentence_list = dataset.train_sentence_list
for sent in input_sentence_list:
    print("系列長:", len(sent), sent.word_list, sent.word_id_list)

系列長: 5 ['素晴らしい', '商品', 'だ', 'と', '思う'] [4, 5, 6, 7, 8]
系列長: 8 ['値段', 'が', '安く', '、', '気兼ね', 'なく', '使い捨て', 'できる'] [9, 10, 11, 12, 13, 14, 15, 16]
系列長: 4 ['いい', '音', 'が', 'する'] [17, 18, 10, 19]
系列長: 5 ['変', 'な', 'におい', 'が', 'する'] [20, 21, 22, 10, 19]
系列長: 6 ['初期', '不良', 'で', '使え', 'なかっ', 'た'] [23, 24, 25, 26, 27, 28]
系列長: 5 ['保障', 'が', 'なく', 'て', '怖い'] [29, 10, 14, 30, 31]


### LMSTによるミニバッチ処理のための前処理

系列長が長い順にソート

In [61]:
import numpy

input_sentence_idx_list = [i for i in range(len(input_sentence_list))]
input_sentence_idx_list.sort(key=lambda x: len(input_sentence_list[x]), reverse=True)
sorted_input_sentence_list = list(numpy.array(input_sentence_list)[input_sentence_idx_list])
length_list = [len(sentence) for sentence in sorted_input_sentence_list]

ミニバッチ処理ではすべてのデータを同様に処理したいので，<br />
系列長を合わせるためにゼロ埋め（パディング）する．<br />
（パディング用のidを決めておけば0でなくても良い．）

In [62]:
import torch
from torch import nn

pad_sentence_list = nn.utils.rnn.pad_sequence([torch.tensor(sentence.word_id_list) for sentence in sorted_input_sentence_list], batch_first=True)
pad_sentence_list

tensor([[ 9, 10, 11, 12, 13, 14, 15, 16],
        [23, 24, 25, 26, 27, 28,  0,  0],
        [ 4,  5,  6,  7,  8,  0,  0,  0],
        [20, 21, 22, 10, 19,  0,  0,  0],
        [29, 10, 14, 30, 31,  0,  0,  0],
        [17, 18, 10, 19,  0,  0,  0,  0]])

### 単語のEmbeddingへの変換とLSTMの入出力

<br />
<img src="figures/batch.jpg" width="320px" align="left"><br clear="all" />
<br />


#### Embedding層の定義

In [63]:
embedding_size = 5
embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)

#### Embedding層への入力

In [64]:
sentence_embeds = embeddings(pad_sentence_list) 

#### padding情報を含んだデータ構造への変換

In [65]:
sentence_embeds = nn.utils.rnn.pack_padded_sequence(sentence_embeds, length_list, batch_first=True) # パディングの情報を含んだLSTMへの入力形式
print(sentence_embeds)

PackedSequence(data=tensor([[-0.4551,  0.0084,  0.4175,  1.9827,  0.6220],
        [-0.5047,  0.7191,  0.2714, -0.3522, -0.9492],
        [-0.0289,  0.6725,  0.9195, -0.6540, -0.1145],
        [-0.8802, -0.4558,  0.9525,  0.9191, -0.5917],
        [-2.4431,  0.0489,  0.0073,  1.3766,  1.3467],
        [ 0.6157, -0.1077,  1.0916,  0.2613,  0.5627],
        [ 0.4615, -0.6179, -1.1043,  0.0164,  2.4451],
        [-0.5915,  0.5454,  0.8148, -0.8063, -0.0991],
        [ 1.4891,  0.8939, -0.1632,  0.5619,  0.8970],
        [ 1.2558, -0.4836,  1.3804, -0.1532,  1.5678],
        [ 0.4615, -0.6179, -1.1043,  0.0164,  2.4451],
        [ 2.5973,  1.1144,  0.2929,  1.9031, -0.8557],
        [ 2.2469, -2.1323,  1.2446, -0.1291, -1.2582],
        [-2.1239,  0.3752, -1.2123,  0.8049,  0.4078],
        [ 0.9385,  0.4654,  1.2850, -0.4962, -0.4349],
        [ 0.4180, -0.1056,  0.8720, -0.1217, -1.0491],
        [-0.6997,  1.0598,  0.2474,  1.6757,  0.6807],
        [ 0.4615, -0.6179, -1.1043,  0.0164, 

#### LSTMの定義

In [66]:
rnn_input_size = embedding_size
rnn_hidden_size = 3
lstm = nn.LSTM(
    rnn_input_size, 
    rnn_hidden_size, 
    num_layers=1, # LSTM_1(LSTM_2((...のようにLSTM を多段にすることができる  今回は一層のみ
    batch_first=True # 入力の形式を指定 (batch_size, 系列長, 分散表現の次元) のようにbatch_sizeが最初に来るよう入力
)

#### LSTMへの入力

In [67]:
lstm_outputs, hidden_cell_tensors = lstm(sentence_embeds) # lstmの計算 各段階での出力 (lstm_outputs) とパディング部分を除いた最終的な出力 (hidden_cell_tensors)が返ってくる

lstm_outputs：各段階での出力 <br />
<br />
<img src="figures/lstm_outputs.jpg" width="320px" align="left"><br clear="all" />
<br />
hidden_cell_tensors[0]：最終状態の出力 <br />
hidden_cell_tensors[1]：最終状態のメモリセル <br />
<br />
<img src="figures/hidden_cell.jpg" width="320px" align="left"><br clear="all" />
<br />

In [68]:
# nn.lstmの各段階での出力の整形とパディング部分をゼロベクトルへの置き換え
lstm_outputs, output_lengths = nn.utils.rnn.pad_packed_sequence(lstm_outputs, batch_first=True)

In [69]:
lstm_outputs

tensor([[[ 0.2708,  0.1403,  0.3248],
         [ 0.2898,  0.1016,  0.0380],
         [-0.4225,  0.0207, -0.0419],
         [-0.3185,  0.0352, -0.0196],
         [-0.0980,  0.0291, -0.0991],
         [ 0.0850,  0.1830,  0.2034],
         [-0.1953,  0.0536,  0.0346],
         [ 0.1697,  0.0711,  0.2374]],

        [[ 0.0183,  0.0691,  0.1739],
         [ 0.0540,  0.0856,  0.2132],
         [ 0.1959,  0.2909,  0.2681],
         [ 0.0793,  0.1587,  0.2252],
         [ 0.3506,  0.2208,  0.4044],
         [ 0.3659, -0.1445,  0.1934],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 0.0549,  0.0474,  0.1131],
         [ 0.1416,  0.0043,  0.0934],
         [ 0.0541,  0.0242,  0.1540],
         [ 0.0380,  0.0543,  0.2076],
         [-0.3362,  0.0312,  0.0168],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000]],

        [[ 0.1773,  0.0777,  0.3446],
         [ 0.2402,  0.0382,  0.1406],
      

In [70]:
hidden_cell_tensors

(tensor([[[ 0.1697,  0.0711,  0.2374],
          [ 0.3659, -0.1445,  0.1934],
          [-0.3362,  0.0312,  0.0168],
          [-0.0696,  0.0558,  0.0663],
          [-0.0951,  0.1760, -0.0564],
          [-0.1407,  0.0464,  0.0804]]], grad_fn=<StackBackward>),
 tensor([[[ 0.2526,  0.1558,  0.7618],
          [ 0.7865, -0.2674,  1.4051],
          [-0.5172,  0.1364,  0.0590],
          [-0.0972,  0.1907,  0.1366],
          [-0.1388,  0.5359, -0.1011],
          [-0.1990,  0.1589,  0.1732]]], grad_fn=<StackBackward>))

### BiLSTMの利用
モデルの定義<br />
<b>bidirectional=True</b> の追加

In [71]:
bilstm = nn.LSTM(
    rnn_input_size, 
    rnn_hidden_size, 
    num_layers=1, # LSTM_1(LSTM_2((...のようにLSTM を多段にすることができる  今回は一層のみ
    batch_first=True, # 入力の形式を指定 (batch_size, 系列長, 分散表現の次元) のようにbatch_sizeが最初に来るよう入力
    bidirectional=True # 双方向LSTMに
)

In [72]:
lstm_outputs, hidden_cell_tensors = bilstm(sentence_embeds) 

In [73]:
# nn.lstmの各段階での出力の整形とパディング部分をゼロベクトルへの置き換え
lstm_outputs, output_lengths = nn.utils.rnn.pad_packed_sequence(lstm_outputs, batch_first=True)

lstm_outputs # 各段階での出力 双方向の各段階での出力を連結 (3 * 2次元)

tensor([[[ 5.0541e-02, -2.4209e-01, -9.3454e-02,  3.7103e-01, -9.2738e-03,
           6.6723e-02],
         [ 5.0225e-03, -6.2451e-01, -1.8603e-01,  3.3411e-01, -6.4509e-03,
           1.6077e-01],
         [ 1.8521e-01, -1.3036e-01, -6.3688e-01,  2.1698e-01,  2.3648e-01,
           6.4992e-02],
         [ 8.7559e-02,  3.2377e-02, -4.4821e-01,  2.1770e-01,  4.9263e-02,
           5.7946e-02],
         [ 1.5802e-01,  1.2415e-01, -5.7881e-01,  1.6962e-01,  1.2508e-01,
           4.1428e-02],
         [ 3.1383e-02, -4.1777e-02, -3.1888e-01,  2.4501e-01,  3.7735e-02,
           4.1011e-02],
         [ 1.4887e-04,  8.7179e-02, -5.4264e-01,  8.5091e-02,  5.9805e-02,
           2.2037e-02],
         [ 1.6317e-01,  2.7392e-02, -4.9134e-01,  4.5514e-02,  1.9806e-01,
           6.4654e-02]],

        [[-1.5590e-01,  2.9880e-02,  3.7991e-02,  2.0814e-01,  1.3574e-02,
          -4.2747e-02],
         [-1.8182e-01,  4.4231e-02,  1.7942e-02,  2.1107e-01, -1.6132e-02,
          -2.3647e-02],
        

In [74]:
# hidden_cell_tensors: 最終的な出力とメモリセル
print(hidden_cell_tensors[0][0]) # 順方向LSTMの出力
print(hidden_cell_tensors[0][1]) # 逆方向LSTMの出力
print(hidden_cell_tensors[1][0]) # 順方向LSTMのメモリセル
print(hidden_cell_tensors[1][1]) # 逆方向LSTMのメモリセル

tensor([[ 0.1632,  0.0274, -0.4913],
        [ 0.0619, -0.2695, -0.3718],
        [ 0.2441,  0.2186, -0.3898],
        [ 0.1334, -0.3703, -0.4355],
        [-0.0322, -0.5984, -0.0865],
        [ 0.1428, -0.3806, -0.4201]], grad_fn=<SelectBackward>)
tensor([[ 0.3710, -0.0093,  0.0667],
        [ 0.2081,  0.0136, -0.0427],
        [ 0.0852,  0.0949, -0.0430],
        [ 0.2140,  0.0085,  0.0255],
        [ 0.4913, -0.1755,  0.0357],
        [ 0.1405,  0.1895,  0.0713]], grad_fn=<SelectBackward>)
tensor([[ 0.1963,  0.0391, -1.7540],
        [ 0.0689, -0.3835, -0.6914],
        [ 0.4870,  0.3829, -0.8055],
        [ 0.2332, -0.5205, -1.5063],
        [-0.1272, -0.8627, -0.9132],
        [ 0.2494, -0.5367, -1.3334]], grad_fn=<SelectBackward>)
tensor([[ 0.4780, -0.0240,  0.1452],
        [ 0.4334,  0.0387, -0.1679],
        [ 0.1899,  0.1902, -0.1436],
        [ 0.2821,  0.0212,  0.1216],
        [ 0.6161, -0.9453,  0.0942],
        [ 0.2407,  0.3193,  0.1846]], grad_fn=<SelectBackward>)
